In [57]:
!python -m spacy download fr_core_news_sm

✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


In [58]:
#Importing required packages
import pandas as pd
import seaborn as sns
import nltk
import matplotlib.pyplot as plt
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import numpy as np
import gensim
from sklearn import datasets
from sklearn.base import TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.utils.multiclass import unique_labels
import string

In [59]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [60]:
#Import data
def remove_punctuations(text):
    punctuations = '\!"#$%&()*+,./:;<=>?@[\]_{|}~'
    for punctuation in punctuations:
        text = text.replace(punctuation, '')
        text = text.replace('-', ' ')
    return text
def remove_digits(text):
    for digits in string.digits:
        text = text.replace(digits, '')
    return text
Sentences = "https://raw.githubusercontent.com/TetraFaal/Big-Scale-Analytics-2021-Tesla/main/Data/Dataset_Aicrowd/22fdbaea-415d-4685-8132-1916959ca359_train.csv"
Sentences = pd.read_csv(Sentences)
Sentences.columns = ['Text','Difficulty']

Sentences["Text"] = Sentences['Text'].apply(remove_punctuations)
Sentences["Text"] = Sentences['Text'].apply(remove_digits)

Cognates = "https://raw.githubusercontent.com/TetraFaal/Big-Scale-Analytics-2021-Tesla/main/Data/Cognates/Cognates.csv"
Cognates = pd.read_csv(Cognates)

CognatesGlossary = "https://raw.githubusercontent.com/TetraFaal/Big-Scale-Analytics-2021-Tesla/main/Data/Cognates/glossaryCognates.csv"
CognatesGlossary = pd.read_csv(CognatesGlossary)

In [61]:
Cognates.head()

,Cognates
0,abattoir
1,abdication
2,abdominal
3,aberrant
4,aberration


In [62]:
CognatesGlossary.head()

,CognatesEN,CognatesFR
0,absolutely,absolument
1,abundance,abondance
2,abuse,abuser
3,accelerate (to),accélérer
4,accentuate (to),accentuer


In [63]:
#Create a cognates list and delete duplicates
ListCo = []
for cognate  in CognatesGlossary.CognatesFR:
  ListCo.append(cognate)
for cognate  in Cognates.Cognates:
  ListCo.append(cognate)
ListCognates = list(set(ListCo))
print(ListCognates)
print(len(ListCognates))

['luminescent', 'micron', 'retirer', 'inhibition', 'condensation', 'substance', 'candide', 'inversion', 'vote\xa0', 'royalties', 'totalement', 'antidote', 'réalité', 'hibernation', 'inverse', 'commensurable', 'picaresque', 'music', 'malachite', 'amplification', 'activité', 'podium', 'délicieuse', 'pertinence', 'classifier', 'honneur', 'carnage', 'contradiction', 'massage', 'orifice', 'apparaître', 'simplifier', 'thymus', 'fameux', 'hormone', 'capturer', 'manipulation', 'poignant', 'prison', 'console', 'combustible', 'irritable', 'rotation', 'examiner', 'indubitable', 'arable', 'cobra', 'innocent', 'vaccination', 'perfection', 'village', 'tempo', 'progression', 'combinaison', 'contagieux', 'destitution', 'phallus', 'tribunal', 'attitude', 'optimal', 'instigation', 'terminer', 'dune', 'modification', 'microphone', 'alligator', 'quintessence', 'miniature', 'art', 'dislocation', 'exact', 'anaconda', 'divergent', 'danger', 'capture', 'ratio', 'posture', 'programme', 'judo', 'sentiment', 'an

In [64]:
pd.set_option("max_colwidth", None)
Sentences.head(10)

,Text,Difficulty
0,C'est pour quand,A1
1,Je pense que c'est bon,A1
2,C'est pas mal,A1
3,Qu'est ce que vous faites,A1
4,C'est bien,A1
5,Ce n'est pas juste,A1
6,Je ne veux pas,A1
7,Je ne sais pas,A1
8,Plus que ça,A1
9,Non je ne le trouve pas,A1


In [65]:
#Calculating base rate (perfect base rate with 6 features is 0.16)
max(len(Sentences[Sentences["Difficulty"] =='A1']) / len(Sentences), len(Sentences[Sentences["Difficulty"] == 'A2']) / len(Sentences), len(Sentences[Sentences["Difficulty"] == 'B1']) / len(Sentences) , len(Sentences[Sentences["Difficulty"] == 'B2']) / len(Sentences),len(Sentences[Sentences["Difficulty"] == 'C1']) / len(Sentences), len(Sentences[Sentences["Difficulty"] == 'C2']) / len(Sentences))

0.16666666666666666

In [66]:
#Counting targets values 
display(Sentences.Difficulty.value_counts())

A1    800
C2    800
A2    800
C1    800
B1    800
B2    800
Name: Difficulty, dtype: int64

In [67]:
indexMax = (Sentences.shape[0] -1)
Sentences.shape[0]

4800

In [68]:
df = Sentences
df = df.assign(Cognates = "")
df.head() 

,Text,Difficulty,Cognates
0,C'est pour quand,A1,
1,Je pense que c'est bon,A1,
2,C'est pas mal,A1,
3,Qu'est ce que vous faites,A1,
4,C'est bien,A1,


In [69]:
sp = spacy.load('fr_core_news_sm')

In [74]:
index = 0
data = list(zip(Sentences["Text"],Sentences["Difficulty"]))
datawords = []
for t, label in data:
  list_of_strings = sp(t)
  token = []
  cognates =""
  for i in list_of_strings:
    token.append(i.text)
    if str(i.text) in ListCognates: 
      token.append(i.text)
      cognates = cognates + " " + str(i.text)
      print(cognates)
  if index <=indexMax:
    df.at[index,'Cognates']= cognates
    index += 1
    print(index)
  datawords.append([token, label])



Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
 classe
2487
 corruption
 corruption exploitation
2488
 plus
 plus plus
 plus plus utiliser
2489
 chaos
2490
2491
 août
2492
 décembre
2493
 participer
 participer transport
2494
2495
2496
 passion
 passion art
 passion art art
2497
2498
2499
2500
2501
 prince
2502
2503
 danger
 danger expérience
2504
 horizon
2505
 minute
2506
2507
2508
2509
 décembre
2510
2511
2512
 occasion
 occasion influence
2513
2514
 information
2515
 usage
2516
 apparition
2517
 apparition
2518
 culture
 culture plus
 culture plus exploitation
2519
 science
2520
2521
 zone
 zone plus
2522
 texture
2523
 plus
 plus plus
 plus plus mode
2524
2525
2526
2527
 plus
2528
2529
2530
 plus
 plus six
2531
 occasion
2532
 classe
2533
2534
2535
 plan
2536
 part
2537
2538
2539
2540
 téléphone
2541
 plus
2542
2543
2544
 lion
2545
 expérience
2546
 utiliser
 utiliser plus
 utiliser plus manière
2547
2548
 colonie
2549
 intrigue
 intrigue image
2550


In [75]:
newdf = pd.DataFrame(datawords, columns=["Text", "Label"])

In [76]:
newdf.head(-1)

,Text,Label
0,"[C', est, pour, quand]",A1
1,"[Je, pense, que, c', est, bon]",A1
2,"[C', est, pas, mal]",A1
3,"[Qu', est, ce, que, vous, faites]",A1
4,"[C', est, bien]",A1
...,...,...
4794,"[Le, jardin, jardin, commença, à, fleurir, après, qu', il, se, fut, mis, à, faire, chaud]",C2
4795,"[Le, problème, problème, va, du, garçon, de, café, tentant, inintelligiblement, de, communiquer, avec, des, touristes, médusés, jusque, comme, on, l', a, vu, aux, niveaux, les, plus, plus, élevés, de, la, société, française]",C2
4796,"[Les, lois, de, programmation, mentionnées, à, l', antépénultième, alinéa, de, l', article, article, , de, la, Constitution, ont, pour, objet, objet, de, déterminer, déterminer, les, objectifs, de, l', action, action, de, l', État]",C2
4797,"[La, conscience, conscience, malheureuse, c', est, donc, le, mauvais, infini, tel, que, la, philosophie, de, Fichte, amène, à, le, concevoir]",C2


In [77]:
df.head(-1)

,Text,Difficulty,Cognates
0,C'est pour quand,A1,
1,Je pense que c'est bon,A1,
2,C'est pas mal,A1,
3,Qu'est ce que vous faites,A1,
4,C'est bien,A1,
...,...,...,...
4794,Le jardin commença à fleurir après qu'il se fut mis à faire chaud,C2,jardin
4795,Le problème va du garçon de café tentant inintelligiblement de communiquer avec des touristes médusés jusque comme on l'a vu aux niveaux les plus élevés de la société française,C2,problème plus
4796,Les lois de programmation mentionnées à l'antépénultième alinéa de l'article de la Constitution ont pour objet de déterminer les objectifs de l'action de l'État,C2,article objet déterminer action
4797,La conscience malheureuse c'est donc le mauvais infini tel que la philosophie de Fichte amène à le concevoir,C2,conscience


In [78]:
FinalData = pd.DataFrame()
FinalData['Text'] = df['Text'].astype('string') + df['Cognates'].astype('string')
FinalData['label'] = df['Difficulty']


,Text,label
0,C'est pour quand,A1
1,Je pense que c'est bon,A1
2,C'est pas mal,A1
3,Qu'est ce que vous faites,A1
4,C'est bien,A1
...,...,...
4795,Le problème va du garçon de café tentant inintelligiblement de communiquer avec des touristes médusés jusque comme on l'a vu aux niveaux les plus élevés de la société française problème plus,C2
4796,Les lois de programmation mentionnées à l'antépénultième alinéa de l'article de la Constitution ont pour objet de déterminer les objectifs de l'action de l'État article objet déterminer action,C2
4797,La conscience malheureuse c'est donc le mauvais infini tel que la philosophie de Fichte amène à le concevoir conscience,C2
4798,Comme Monsieur un informaticien fondateur de la librairie des maths pour qui il y a tellement de manières d'aimer les mathématiques qu'il propose de nombreux produits plus ou moins littéraires plus ou moins ludiques plus plus,C2


In [79]:
FinalData.head(-1)

,Text,label
0,C'est pour quand,A1
1,Je pense que c'est bon,A1
2,C'est pas mal,A1
3,Qu'est ce que vous faites,A1
4,C'est bien,A1
...,...,...
4794,Le jardin commença à fleurir après qu'il se fut mis à faire chaud jardin,C2
4795,Le problème va du garçon de café tentant inintelligiblement de communiquer avec des touristes médusés jusque comme on l'a vu aux niveaux les plus élevés de la société française problème plus,C2
4796,Les lois de programmation mentionnées à l'antépénultième alinéa de l'article de la Constitution ont pour objet de déterminer les objectifs de l'action de l'État article objet déterminer action,C2
4797,La conscience malheureuse c'est donc le mauvais infini tel que la philosophie de Fichte amène à le concevoir conscience,C2


In [81]:
display(FinalData.shape, FinalData.info(), FinalData.describe)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4800 entries, 0 to 4799
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    4800 non-null   string
 1   label   4800 non-null   object
dtypes: object(1), string(1)
memory usage: 75.1+ KB


(4800, 2)

None

<bound method NDFrame.describe of                                                                                                                                                                                                                                    Text label
0                                                                                                                                                                                                                     C'est pour quand     A1
1                                                                                                                                                                                                                Je pense que c'est bon    A1
2                                                                                                                                                                                                                         C'est pas mal    A1
3             

In [83]:
FinalData.to_csv('Data_whitout_ponctuation_digits_with_cognates.csv', encoding='utf-8-sig', index=False)